<a href="https://colab.research.google.com/github/samkitkankariya/Article-Text-Extraction-and-NLP-Analysis/blob/main/Article_Extraction_NLP_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Read the Excel file
df = pd.read_excel('Input.xlsx')

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']

    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the text content from the HTML
        heading = soup.find('h1', class_ = 'entry-title')
        paragraphs = soup.find('div', class_ = 'td-post-content tagdiv-type')

        # Save the content to a text file
        with open(f'Extracted Files Data/{url_id}.txt', 'w') as f:
            f.write(heading.text)
            for p in paragraphs:
                f.write(p.text)

        print(f"Data scraped successfully for URL_ID: {url_id}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for URL_ID: {url_id} - {e}")

    except Exception as e:
        print(f"An error occurred for URL_ID: {url_id} - {e}")



Data scraped successfully for URL_ID: blackassign0001
Data scraped successfully for URL_ID: blackassign0002
Data scraped successfully for URL_ID: blackassign0003
Data scraped successfully for URL_ID: blackassign0004
Data scraped successfully for URL_ID: blackassign0005
Data scraped successfully for URL_ID: blackassign0006
Data scraped successfully for URL_ID: blackassign0007
Data scraped successfully for URL_ID: blackassign0008
Data scraped successfully for URL_ID: blackassign0009
Data scraped successfully for URL_ID: blackassign0010
Data scraped successfully for URL_ID: blackassign0011
Data scraped successfully for URL_ID: blackassign0012
Data scraped successfully for URL_ID: blackassign0013
An error occurred for URL_ID: blackassign0014 - 'NoneType' object has no attribute 'text'
Data scraped successfully for URL_ID: blackassign0015
Data scraped successfully for URL_ID: blackassign0016
Data scraped successfully for URL_ID: blackassign0017
Data scraped successfully for URL_ID: blackass

In [ ]:
# List of indices to scrape
# For the files which had error  - 'NoneType' object has no attribute 'text'
indices_to_scrape = [13, 19, 28, 42,82,83,91,98,99]  # For the files which had error

for index in indices_to_scrape:
    url_id = df.iloc[index]['URL_ID']
    url = df.iloc[index]['URL']

    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the text content from the HTML
        heading = soup.h1.text
        paragraphs = soup.find('div', class_ = 'td_block_wrap tdb_single_content tdi_130 td-pb-border-top td_block_template_1 td-post-content tagdiv-type')
        p1 = paragraphs.div
        # Save the content to a text file
        with open(f'Extracted Files Data/{url_id}.txt', 'w') as f:
            f.write(heading)
            for p in p1:
                f.write(p1.p.text)

        print(f"Data scraped successfully for URL_ID: {url_id}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for URL_ID: {url_id} - {e}")

    except Exception as e:
        print(f"An error occurred for URL_ID: {url_id} - {e}")



Data scraped successfully for URL_ID: blackassign0014
Data scraped successfully for URL_ID: blackassign0020
Data scraped successfully for URL_ID: blackassign0029
Data scraped successfully for URL_ID: blackassign0043
Data scraped successfully for URL_ID: blackassign0083
Data scraped successfully for URL_ID: blackassign0084
Data scraped successfully for URL_ID: blackassign0092
Data scraped successfully for URL_ID: blackassign0099
Data scraped successfully for URL_ID: blackassign0100


In [ ]:
import pandas as pd
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import re
import io
nltk.download('punkt')  # Ensure NLTK tokenizer is downloaded

# Load stop words
stop_words_folder = 'StopWords'
stop_words = []
for file_name in os.listdir(stop_words_folder):
    with open(os.path.join(stop_words_folder, file_name), 'r',  encoding='latin-1') as f:
        stop_words.extend(f.read().splitlines())

# Load positive and negative word dictionaries
positive_words_path = 'MasterDictionary/positive-words.txt'
negative_words_path = 'MasterDictionary/negative-words.txt'

def load_word_list(file_path, encoding='utf-8'):
    with io.open(file_path, 'r', encoding=encoding) as file:
        return file.read().splitlines()

positive_words = load_word_list(positive_words_path, encoding='latin-1')
negative_words = load_word_list(negative_words_path, encoding='latin-1')


# Define a function for sentiment analysis and derived variables calculation
def analyze_sentiment(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Perform sentiment analysis based on positive and negative dictionaries
    positive_score = sum(1 for token in tokens if token.lower() in positive_words)
    negative_score = sum(1 for token in tokens if token.lower() in negative_words)

    # Calculate Polarity Score
    polarity_score = (positive_score - negative_score) / (positive_score + negative_score + 0.000001)

    # Calculate Subjectivity Score
    subjectivity_score = (positive_score + negative_score) / (len(tokens) + 0.000001)

    # Calculate readability metrics
    sentences = nltk.sent_tokenize(text)
    total_words = len(tokens)
    total_sentences = len(sentences)
    avg_sentence_length = total_words / total_sentences
    complex_words = sum(1 for token in tokens if len(token) > 2)  # Assuming words with more than 2 syllables are complex
    percentage_complex_words = complex_words / total_words
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    avg_words_per_sentence = total_words / total_sentences

    # Count syllables
    def count_syllables(word):
        count = 0
        vowels = 'aeiouy'
        word = word.lower()
        if word.endswith(('es', 'ed')):
            pass
        else:
            for index, letter in enumerate(word):
                if letter in vowels and (index == 0 or word[index - 1] not in vowels):
                    count += 1
        return count

    syllables_count = sum(count_syllables(token) for token in tokens)

    # Count personal pronouns
    personal_pronouns = ['i', 'we', 'my', 'ours', 'us']
    personal_pronouns_count = sum(1 for token in tokens if token.lower() in personal_pronouns)

    # Calculate average word length
    avg_word_length = sum(len(token) for token in tokens) / total_words

    return positive_score, negative_score, polarity_score, subjectivity_score, avg_sentence_length, \
           percentage_complex_words, fog_index, avg_words_per_sentence, syllables_count, personal_pronouns_count, avg_word_length

# Create a DataFrame for storing output data
output_df = pd.DataFrame(columns=['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE',
                                   'POLARITY SCORE', 'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH',
                                   'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE',
                                   'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'])

# Iterate over each URL_ID and perform sentiment analysis and derived variables calculation
for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']

    try:
        # Read the text file content
        with open(f'Extracted Files Data/{url_id}.txt', 'r') as text_file:
            text = text_file.read()

        # Perform sentiment analysis and derived variables calculation
        positive_score, negative_score, polarity_score, subjectivity_score, avg_sentence_length, \
        percentage_complex_words, fog_index, avg_words_per_sentence, syllables_count, personal_pronouns_count, avg_word_length \
        = analyze_sentiment(text)

        # Append the results to the DataFrame
        new_row = ({'URL_ID': url_id, 'URL': url,'POSITIVE SCORE': positive_score,
                                      'NEGATIVE SCORE': negative_score, 'POLARITY SCORE': polarity_score,
                                      'SUBJECTIVITY SCORE': subjectivity_score, 'AVG SENTENCE LENGTH': avg_sentence_length,
                                      'PERCENTAGE OF COMPLEX WORDS': percentage_complex_words, 'FOG INDEX': fog_index,
                                      'AVG NUMBER OF WORDS PER SENTENCE': avg_words_per_sentence,
                                      'SYLLABLE PER WORD': syllables_count, 'PERSONAL PRONOUNS': personal_pronouns_count,
                                      'AVG WORD LENGTH': avg_word_length})

        output_df.loc[len(output_df)] = new_row

    except FileNotFoundError:
        print(f"File not found for URL_ID: {url_id}")

# Save the DataFrame to 'Output.xlsx'
output_df.to_excel('Output Data Structure Self.xlsx', index=False)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


File not found for URL_ID: blackassign0036
File not found for URL_ID: blackassign0049
